In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim

%matplotlib inline

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#import NDC deliveries csv
ndc_partner_del = pd.read_csv('../data/ndc_deliveries.csv')
print(ndc_partner_del.shape)
ndc_partner_del.head()

In [ ]:
#get total deliveries by county
total_by_county=ndc_partner_del.groupby(['County'], as_index=False)['Total'].sum()
total_by_county

In [ ]:
total_by_county.to_csv('../files/total_by_county.csv', index = False)

In [ ]:
#import county file
counties = gpd.read_file('../data/county.geojson')
counties.head()

In [ ]:
# keep only rows for counties needed
counties=counties[counties['COUNTY_STATE_CODE'].isin(['Davidson, TN','Wilson, TN','Williamson, TN','Washington, TN','Marshall, TN','Maury, TN'])]
counties.head()

In [ ]:
#create TN counties
polygon_tn = counties
polygon_tn.shape

In [ ]:
ndc_map = folium.Map(
    location=[35.85,-86.767960],
    zoom_start=8.5,
    #tiles='Stamen Terrain'
)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(ndc_map)

folium.GeoJson(polygon_tn).add_to(ndc_map)

folium.Choropleth(
    geo_data=counties,
    name='choropleth',
    data=total_by_county,
    columns=['County', 'Total'],
    key_on='feature.properties.COUNTY_STATE_CODE',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Diapers Distributed to Partners 2020 YTD'
).add_to(ndc_map)

folium.LayerControl().add_to(ndc_map)

# inside the loop add each marker to the cluster
#for row_index, row_values in ndc_partner_del.iterrows():
#    loc = [row_values['Latitude'], row_values['Longitude']]
#    pop = '<b>Partner:</b> ' + str(row_values['Partners']) + ' ' + '<b>Diapers Delivered:</b> ' + str(row_values['Total'])
#    icon=folium.Icon(color="orange",icon="circle", prefix='fa')
    
#    marker = folium.Marker(
#        location = loc, 
#        popup = pop,
#    icon = icon) 
    
#    marker.add_to(marker_cluster)
    
#for pop ups
for ind, row in ndc_partner_del.iterrows(): 
    point = [row['Latitude'], row['Longitude']]
    popup = '<b>Partner:</b> ' + str(row['Partners']) + ' ' + '<b>Diapers Delivered:</b> ' + str(row['Total'])
    folium.CircleMarker(point, popup = popup, radius=5, fill=True, color='orange').add_to(ndc_map)
    
    #now color format based on the strength of the tornado
    #if row['EF_Scale']=="EF3":
    #    folium.CircleMarker(point, popup = popup, radius=5, fill=True, color='red').add_to(new_map_911_tornado)
    #else:
    #    folium.CircleMarker(point, popup = popup, radius=5, fill=True, color='orange').add_to(new_map_911_tornado)
    
ndc_map

In [ ]:
#save to html
#ndc_map.save('../files/ndc_map.html')

In [ ]:
#save as png
ndc_map.save('../files/ndc_map.png',bbox_inches='tight', dpi=300)